In this notebook we load a saved dreamer, and run it, to look at params, speed and improve hackability

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# TODO make this a proper package
import os, sys
sys.path.append('..')


from dreamer import parse_args, main, make_env, make_dataset, count_steps,Dreamer

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Loading textures from cache


In [3]:
# emulate cli
argv = f"../dreamer.py --configs craftax_small --logdir ../logdir/craftax_small"
argv = argv.split()
print(argv)
config = parse_args(argv)
config

['../dreamer.py', '--configs', 'craftax_small', '--logdir', '../logdir/craftax_small']


Namespace(act='SiLU', action_repeat=1, actor={'layers': 3, 'dist': 'onehot', 'entropy': 0.0003, 'unimix_ratio': 0.01, 'std': 'none', 'min_std': 0.1, 'max_std': 1.0, 'temp': 0.1, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 1.0}, batch_length=16, batch_size=128, compile=True, cont_head={'layers': 3, 'loss_scale': 1.0, 'outscale': 1.0}, critic={'layers': 2, 'dist': 'symlog_disc', 'slow_target': True, 'slow_target_update': 1, 'slow_target_fraction': 0.02, 'lr': 3e-05, 'eps': 1e-05, 'grad_clip': 100.0, 'outscale': 0.0}, dataset_size=1000000, debug=False, decoder={'mlp_keys': 'state', 'cnn_keys': '$^', 'act': 'SiLU', 'norm': True, 'cnn_depth': 32, 'kernel_size': 4, 'minres': 4, 'mlp_layers': 3, 'mlp_units': 256, 'cnn_sigmoid': False, 'image_dist': 'mse', 'vector_dist': 'symlog_mse', 'outscale': 1.0}, deterministic_run=False, device='cuda:0', disag_action_cond=False, disag_layers=4, disag_log=True, disag_models=10, disag_offset=1, disag_target='stoch', disag_units=400, discount

In [4]:
from loguru import logger
from tqdm.auto import tqdm
import pathlib

import torch
from torch import nn
from torch import distributions as torchd

import exploration as expl
import models
import tools
import envs.wrappers as wrappers
from parallel import Parallel, Damy

# from main
tools.set_seed_everywhere(config.seed)
if config.deterministic_run:
    tools.enable_deterministic_run()
logdir = pathlib.Path(config.logdir).expanduser()
config.traindir = config.traindir or logdir / "train_eps"
config.evaldir = config.evaldir or logdir / "eval_eps"
config.steps //= config.action_repeat
config.eval_every //= config.action_repeat
config.log_every //= config.action_repeat
config.time_limit //= config.action_repeat

logger.info(f"Logdir {logdir}")
logdir.mkdir(parents=True, exist_ok=True)
config.traindir.mkdir(parents=True, exist_ok=True)
config.evaldir.mkdir(parents=True, exist_ok=True)
step = count_steps(config.traindir)
# step in logger is environmental step
tlogger = tools.Logger(logdir, config.action_repeat * step)
logger.add(logdir/"logger.log")

logger.info("Create envs.")
if config.offline_traindir:
    directory = config.offline_traindir.format(**vars(config))
else:
    directory = config.traindir
train_eps = tools.load_episodes(directory, limit=config.dataset_size)
if config.offline_evaldir:
    directory = config.offline_evaldir.format(**vars(config))
else:
    directory = config.evaldir
eval_eps = tools.load_episodes(directory, limit=1)
make = lambda mode, id: make_env(config, mode, id)
train_envs = [make("train", i) for i in range(config.envs)]
eval_envs = [make("eval", i) for i in range(config.envs)]
if config.parallel:
    train_envs = [Parallel(env, "process") for env in train_envs]
    eval_envs = [Parallel(env, "process") for env in eval_envs]
else:
    train_envs = [Damy(env) for env in train_envs]
    eval_envs = [Damy(env) for env in eval_envs]
acts = train_envs[0].action_space
logger.info(f"Action Space {acts}" )
config.num_actions = acts.n if hasattr(acts, "n") else acts.shape[0]

state = None
if not config.offline_traindir:
    prefill = max(0, config.prefill - count_steps(config.traindir))
    logger.info(f"Prefill dataset ({prefill} steps).")
    if hasattr(acts, "discrete"):
        random_actor = tools.OneHotDist(
            torch.zeros(config.num_actions).repeat(config.envs, 1)
        )
    else:
        random_actor = torchd.independent.Independent(
            torchd.uniform.Uniform(
                torch.Tensor(acts.low).repeat(config.envs, 1),
                torch.Tensor(acts.high).repeat(config.envs, 1),
            ),
            1,
        )

    def random_agent(o, d, s):
        action = random_actor.sample()
        logprob = random_actor.log_prob(action)
        return {"action": action, "logprob": logprob}, None

    state = tools.simulate(
        random_agent,
        train_envs,
        train_eps,
        config.traindir,
        tlogger,
        limit=config.dataset_size,
        steps=prefill,
    )
    tlogger.step += prefill * config.action_repeat
    logger.info(f"Logger: ({tlogger.step} steps).")

logger.info("Simulate agent.")
train_dataset = make_dataset(train_eps, config)
eval_dataset = make_dataset(eval_eps, config)

2024-06-06 13:35:50.147 | INFO     | __main__:<module>:27 - Logdir ../logdir/craftax_small
2024-06-06 13:35:50.153 | INFO     | __main__:<module>:36 - Create envs.
2024-06-06 13:36:42.176 | INFO     | __main__:<module>:57 - Action Space Box(0.0, 1.0, (43,), float32)
2024-06-06 13:36:42.178 | INFO     | __main__:<module>:63 - Prefill dataset (0 steps).
2024-06-06 13:36:42.180 | INFO     | __main__:<module>:92 - Logger: (128521 steps).
2024-06-06 13:36:42.180 | INFO     | __main__:<module>:94 - Simulate agent.


In [7]:
config = parse_args(argv)
config.num_actions = acts.n if hasattr(acts, "n") else acts.shape[0]
agent = Dreamer(
    train_envs[0].observation_space,
    train_envs[0].action_space,
    config,
    tlogger,
    train_dataset,
).to(config.device)
# print(agent)
agent.requires_grad_(requires_grad=False)
if (logdir / "latest.pt").exists():
    checkpoint = torch.load(logdir / "latest.pt")
    agent.load_state_dict(checkpoint["agent_state_dict"])
    tools.recursively_load_optim_state_dict(agent, checkpoint["optims_state_dict"])
    agent._should_pretrain._once = False
    logger.warning(f"Loaded model from {logdir / 'latest.pt'}")

2024-06-06 13:38:20.651 | INFO     | networks:__init__:323 - Encoder CNN shapes: {}
2024-06-06 13:38:20.651 | INFO     | networks:__init__:324 - Encoder MLP shapes: {'state': (16536,)}
2024-06-06 13:38:20.751 | INFO     | networks:__init__:390 - Decoder CNN shapes: {}
2024-06-06 13:38:20.751 | INFO     | networks:__init__:391 - Decoder MLP shapes: {'state': (16536,)}
2024-06-06 13:38:20.813 | INFO     | models:__init__:102 - Optimizer model_opt has 15732120 variables.
2024-06-06 13:38:20.836 | INFO     | models:__init__:281 - Optimizer actor_opt has 1335851 variables.
2024-06-06 13:38:20.837 | INFO     | models:__init__:292 - Optimizer value_opt has 1181439 variables.
2024-06-06 13:38:21.032 | WARNING  | __main__:<module>:17 - Loaded model from ../logdir/craftax_small/latest.pt


- note model_opt includes actor.wm
  - encoder
  - rssm
  - heads
- actor

## Check encoder decoder

In [ ]:
agent.encoder(x)
agent.heads['decoder'](x)   

## Now lets play

In [8]:
assert state is not None
import numpy as np

state

(0, 0, array([ True]), array([0], dtype=int32), [None], None, [0])

In [9]:
from tools import convert, add_to_cache
envs = train_envs
cache = train_eps

step, episode = 0, 0
done = np.ones(len(envs), bool)
length = np.zeros(len(envs), np.int32)
obs = [None] * len(envs)
agent_state = None
reward = [0] * len(envs)

indices = [index for index, d in enumerate(done) if d]
results = [envs[i].reset() for i in indices]
results = [r() for r in results]
for index, result in zip(indices, results):
    t = result.copy()
    t = {k: convert(v) for k, v in t.items()}
    # action will be added to transition in add_to_cache
    t["reward"] = 0.0
    t["discount"] = 1.0
    # initial state should be added to cache
    add_to_cache(cache, envs[index].id, t)
    # replace obs with done by initial state
    obs[index] = result
# step agents

In [10]:
# from tools.simulate

# step
# step, episode, done, length, obs, agent_state, reward = state
obs = {k: np.stack([o[k] for o in obs]) for k in obs[0] if "log_" not in k}
action, agent_state = agent(obs, done, agent_state)

2024-06-06 13:38:34.000 | INFO     | tools:write:85 - [128521] model_loss 22.2 / model_grad_norm 14.4 / state_loss 17.4 / reward_loss 0.1 / cont_loss 0.0 / kl_free 1.0 / dyn_scale 0.5 / rep_scale 0.1 / dyn_loss 7.8 / rep_loss 7.8 / kl 7.7 / prior_ent 48.4 / post_ent 40.7 / normed_target_mean 0.4 / normed_target_std 0.3 / normed_target_min -0.3 / normed_target_max 1.8 / EMA_005 12.3 / EMA_095 26.4 / value_mean 18.2 / value_std 4.3 / value_min 10.1 / value_max 31.1 / target_mean 18.4 / target_std 4.7 / target_min 8.4 / target_max 37.8 / imag_reward_mean 0.0 / imag_reward_std 0.1 / imag_reward_min -0.2 / imag_reward_max 1.0 / imag_action_mean 10.0 / imag_action_std 12.9 / imag_action_min 0.0 / imag_action_max 42.0 / actor_entropy 0.9 / actor_loss 0.1 / actor_grad_norm 0.5 / value_loss 1.3 / value_grad_norm 0.9 / update_count 1.0 / fps 0.0


In [40]:
from torchinfo import summary

summary(agent, input=(obs, done, agent_state), depth=4)

Layer (type:depth-idx)                                       Param #
Dreamer                                                      --
├─OptimizedModule: 1-1                                       --
│    └─WorldModel: 2-1                                       --
│    │    └─MultiEncoder: 3-1                                --
│    │    │    └─MLP: 4-1                                    (4,365,824)
│    │    └─RSSM: 3-2                                        512
│    │    │    └─Sequential: 4-2                             (547,328)
│    │    │    └─GRUCell: 4-3                                (1,575,936)
│    │    │    └─Sequential: 4-4                             (263,168)
│    │    │    └─Sequential: 4-5                             (394,240)
│    │    │    └─Linear: 4-6                                 (525,312)
│    │    │    └─Linear: 4-7                                 (525,312)
│    │    └─ModuleDict: 3-3                                  --
│    │    │    └─MultiDecoder: 4-8           

In [36]:
# agent._wm.heads

## Fine grained torchinfo

In [12]:
wm = agent._wm
data = next(agent._dataset) 
# self._train()
# post, context, mets = wm._train(data)
data = wm.preprocess(data)
embed = wm.encoder(data)
post, prior = wm.dynamics.observe(
    embed, data["action"], data["is_first"]
)

In [41]:
summary(wm.encoder, input_data=(data,), depth=4, col_names=["input_size", "output_size", "num_params", ])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MultiEncoder                             [128, 16, 130, 110, 3]    [128, 16, 256]            --
├─MLP: 1-1                               [128, 16, 16536]          [128, 16, 256]            --
│    └─Sequential: 2-1                   [128, 16, 16536]          [128, 16, 256]            --
│    │    └─Linear: 3-1                  [128, 16, 16536]          [128, 16, 256]            (4,233,216)
│    │    └─LayerNorm: 3-2               [128, 16, 256]            [128, 16, 256]            (512)
│    │    └─SiLU: 3-3                    [128, 16, 256]            [128, 16, 256]            --
│    │    └─Linear: 3-4                  [128, 16, 256]            [128, 16, 256]            (65,536)
│    │    └─LayerNorm: 3-5               [128, 16, 256]            [128, 16, 256]            (512)
│    │    └─SiLU: 3-6                    [128, 16, 256]            [128, 16, 256]            --
│    │    └─Li

In [42]:
# heads
feat = wm.dynamics.get_feat(post)
for name, head in wm.heads.items():
    try:
        o = summary(head, input_data=(feat,), depth=3, col_names=["input_size", "output_size", "num_params", ])
        print(name)
        print(o)
    except Exception as e:
        print(f"Summary Failed for {name} {e}")
        continue

decoder
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MultiDecoder                             [128, 16, 1536]           --                        --
├─MLP: 1-1                               [128, 16, 1536]           --                        --
│    └─Sequential: 2-1                   [128, 16, 1536]           [128, 16, 256]            --
│    │    └─Linear: 3-1                  [128, 16, 1536]           [128, 16, 256]            (393,216)
│    │    └─LayerNorm: 3-2               [128, 16, 256]            [128, 16, 256]            (512)
│    │    └─SiLU: 3-3                    [128, 16, 256]            [128, 16, 256]            --
│    │    └─Linear: 3-4                  [128, 16, 256]            [128, 16, 256]            (65,536)
│    │    └─LayerNorm: 3-5               [128, 16, 256]            [128, 16, 256]            (512)
│    │    └─SiLU: 3-6                    [128, 16, 256]            [128, 16, 256]            --
│    │  

In [15]:
# fail as no call method
# summary(wm.dynamics, input_data=(embed, data["action"], data["is_first"]), depth=3, col_names=["output_size", "num_params", ])

In [32]:
actor = agent._task_behavior.actor

summary(actor.layers, input_data=(feat,), depth=3, col_names=["output_size", "num_params", "output_size" ])



Layer (type:depth-idx)                   Output Shape              Param #                   Output Shape
Sequential                               [128, 16, 512]            --                        [128, 16, 512]
├─Linear: 1-1                            [128, 16, 512]            (786,432)                 [128, 16, 512]
├─LayerNorm: 1-2                         [128, 16, 512]            (1,024)                   [128, 16, 512]
├─SiLU: 1-3                              [128, 16, 512]            --                        [128, 16, 512]
├─Linear: 1-4                            [128, 16, 512]            (262,144)                 [128, 16, 512]
├─LayerNorm: 1-5                         [128, 16, 512]            (1,024)                   [128, 16, 512]
├─SiLU: 1-6                              [128, 16, 512]            --                        [128, 16, 512]
├─Linear: 1-7                            [128, 16, 512]            (262,144)                 [128, 16, 512]
├─LayerNorm: 1-8              

In [33]:
value = agent._task_behavior.actor
summary(value.layers, input_data=(feat,), depth=3, col_names=["output_size", "num_params", "output_size" ])

Layer (type:depth-idx)                   Output Shape              Param #                   Output Shape
Sequential                               [128, 16, 512]            --                        [128, 16, 512]
├─Linear: 1-1                            [128, 16, 512]            (786,432)                 [128, 16, 512]
├─LayerNorm: 1-2                         [128, 16, 512]            (1,024)                   [128, 16, 512]
├─SiLU: 1-3                              [128, 16, 512]            --                        [128, 16, 512]
├─Linear: 1-4                            [128, 16, 512]            (262,144)                 [128, 16, 512]
├─LayerNorm: 1-5                         [128, 16, 512]            (1,024)                   [128, 16, 512]
├─SiLU: 1-6                              [128, 16, 512]            --                        [128, 16, 512]
├─Linear: 1-7                            [128, 16, 512]            (262,144)                 [128, 16, 512]
├─LayerNorm: 1-8              

In [62]:
o = obs['state'].reshape((-1, 8268))
map = o[:, :8217].reshape((-1, 9, 11, 83))
map.shape
inventories = o[:, 8217:]
inventories

map

array([[[[0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.]],

        [[0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         ...,
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.]],

        [[0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         ...,
         [0., 0., 1., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         [0., 0., 0., ..., 0., 0., 1.],
         ...,
         [0., 0., 1., ..., 0., 0., 1.],
         [0., 0., 1., ..., 0., 0., 1.],
    

8268